## Tutorial for using the python bindings

In [5]:
from mcm_py import Data, Model, MCMSearch
import numpy as np

### Partition

The partition of a set of variables can be represented in two different ways.
- The first option is to use a 2D binary numpy array. In this form each row represents an ICC and each column a variable. <br>
If a variable is present in a component, the corresponding element is one and zero otherwise. To give an example: 

    `partition = np.array([[1,0,1,0,1,1,0,1,0], [0,1,0,1,0,0,0,0,0], [0,0,0,0,0,0,1,0,1]])`

    This is a partition of 9 variables. The first component contains variables 0, 2, 4, 5 and 7. <br>
    The second components consists of variables 1 and 3. Variables 6 and 8 are present in the third component.

- The second option is to use a 1D numpy array with values between 0 and n-1. In this form the elements correspond to the component index in which each variable is present. To indicate that a variable is not present in the model, the component index `-1` is used.<br>
The same partition as in the example in this form would be:

    `partition = np.array([0,1,0,1,0,0,2,0,2])`

### Data

The Data class is responsible for storing the dataset and provides several statistical methods.  

#### Initialization

An object of this class is initialized with:
- `filename`: the path to the file containing the dataset
- `n`: the number of variables in the system (the maximum system size is 128)
- `q`: the number of states each variable can take


In [2]:
n = 9
q = 2
data = Data('../input/US_SupremeCourt_n9_N895.dat', n, q)

print("The dataset contains %i datapoints, of which %i different ones." %(data.N, data.N_unique))

The dataset contains 895 datapoints, of which 128 different ones.


#### Entropy

One statistical property that we can get is the entropy of the dataset. The default logarithmic base that is used in the calculation of the entropy is base $q$.

In [3]:
data.entropy()

4.411845516595921

In [4]:
data.entropy(base=10)

1.3280978367310246

#### Log-evidence

The Data object also contains methods to calculate the log-evidence of a given partition of the variables:
- `.log_evidence()` returns the total log-evidence of the partition.
- `.log_evidence_icc()` returns the log-evidence of each ICC as an array.

Both functions take as an argument a partition of the variables

NOTE that no check is done to see if the given partition is valid. <br>
In the next section, we discuss Model objects. These can also be used as argument in these methods and are guarenteed to have a valid partition.

In [5]:
partition_array = np.array([[1,0,1,0,1,1,0,1,0], [0,1,0,1,0,0,0,0,0], [0,0,0,0,0,0,1,0,1]])
data.log_evidence(partition_array)

-4121.7115195691

In [6]:
partition_gray_code = np.array([0,1,0,1,0,0,2,0,2])
data.log_evidence(partition_gray_code)

-4121.7115195691

In [7]:
data.log_evidence_icc(partition_array)

array([-1934.44276091, -1093.07714883, -1094.19160984])

In [8]:
data.log_evidence_icc(partition_gray_code)

array([-1934.44276091, -1093.07714883, -1094.19160984])

### Model

The Model class contains a partition of $n$ variables, some of its properties and methods to manipulate the partition.

#### Initialization

An object of this class has several initialization options.<br><br>
Option 1:
- `n`: the number of variables in the system (the maximum system size is 128). The independent model is used as the default partition.

Option 2:
- `n`: the number of variables in the system (the maximum system size is 128).
- `partition`: A specific partition of the n variables given as a numpy array.

Option 3:
- `n`: the number of variables in the system (the maximum system size is 128).
- `partition`: A partition type given as a string. Options are:
    - "independent": all variables in a separate component
    - "complete": all variables in a single component
    - "random": each variable is assigned to a random component

The partition of the model can be returned as a 2D binary numpy array using `.array`

In [9]:
model = Model(n)
model.array

array([[1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=int8)

In [10]:
model.array_gray_code

array([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype=int8)

In [11]:
model = Model(n, partition_array)
model.array

array([[1, 0, 1, 0, 1, 1, 0, 1, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 1]], dtype=int8)

In [12]:
model.array_gray_code

array([0, 1, 0, 1, 0, 0, 2, 0, 2], dtype=int8)

As mentioned in the previous section, this can be used as argument in the function to calculate the log-evidence

In [13]:
data.log_evidence(model)

-4121.7115195691

In [14]:
data.log_evidence_icc(model)

array([-1934.44276091, -1093.07714883, -1094.19160984])

In [15]:
model2 = Model(n, "random")
print("The %i variables are divided into %i components" %(model2.n, model2.n_icc))
model2.array

The 9 variables are divided into 5 components


array([[0, 0, 0, 1, 1, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 1, 0, 0, 0]], dtype=int8)

The partition in the array can be reset by giving a new partition as a 2D binary numpy array.

In [16]:
model.array = model2.array
model.array

array([[0, 0, 0, 1, 1, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 1, 0, 0, 0]], dtype=int8)

Note that element-wise manipulation of model.array doesn't change the partition inside the model.
Manipulation of the partition can be done using the built-in methods:
- `move_variable_in(var_index, comp_index)` : Add the variable `var_index` to component `comp_index`
- `move_variable_out(var_index)` : Remove the variable `var_index`
- `move_varialbe(var_index, comp_index)` : Move the variable `var_index` to component `comp_index`

In [17]:
model = Model(n, "complete")
model.array

array([[1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int8)

In [18]:
model.move_variable_out(0)
model.array

array([[0, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int8)

In [19]:
model.move_variable(7,1)
model.array

array([[0, 1, 1, 1, 1, 1, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 0]], dtype=int8)

In [20]:
model.move_variable_in(0, 2)
model.array

array([[0, 1, 1, 1, 1, 1, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int8)

Note that the Model class contains several other members related to the best log-evidence. However, these are only accesible if the object is the result of a search (see next section). 

In [21]:
model.is_optimized

False

In [22]:
model.get_best_log_evidence()

RuntimeError: No search has been ran yet.

### MCMSearch

The MCMSearch class is responsible for finding the best partition of the variables for a given dataset.
It has to be initialized without any arguments and contains several search methods: greedy search, divide and conquer, simulated annealing and exhaustive search.

In [2]:
searcher = MCMSearch()

#### Greedy search

The greedy search method consists of an iterative merging procedure.
It calculates how much the log-evidence would change if any two components would merge.
The merging of two components that leads to the largest increase in log-evidence is chosen as the starting point in the next iteration. If none of the mergings results in an improved log-evidence, the algorithm stops.

The methods requires a Data object as argument and optionally a Model object from which the partition is used as the starting partition of the algorithm.
If no Model object is given, the independent model is chosen as the starting partition.

In [24]:
final_model = searcher.greedy_search(data)

In [25]:
init_model = searcher.get_model_in()

In [26]:
print("The starting partition of the algorithm is:")
init_model.array

The starting partition of the algorithm is:


array([[1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=int8)

In [27]:
print("The optimal partition of the variables found by the greedy search is:")
final_model.array

The optimal partition of the variables found by the greedy search is:


array([[1, 0, 1, 1, 1, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 1, 0, 1, 1]], dtype=int8)

In [28]:
final_model.get_best_log_evidence()

-3300.395469673639

In [29]:
final_model.get_best_log_evidence_icc()

array([-1754.41166022, -1545.98380945])

#### Divide and conquer

The divide and conquer method consists of a recursive splitting process.
Sequantially each variable is moved to a new component. From all possible splits, the division that results in the largest increase in the log-evidence or the smallest decrease if no division results in an increase, is selected as the starting point in the next iteration.
The process continues until all but one variables are moved and at the end we backtrack to the partition that had the largest log-evidence.
If no split increased the log-evidence, the algorithm stops and otherwise the division procedure is repeated on both components separately.

The methods requires a Data object as argument and optionally a Model object from which the partition is used as the starting partition of the algorithm.
If no Model object is given, the complete model is chosen as the starting partition.

In [48]:
init_model = Model(n, "complete")
final_model = searcher.divide_and_conquer(data, init_model)

In [45]:
print("The optimal partition of the variables found by the divide and conquer method is:")
final_model.array

The optimal partition of the variables found by the divide and conquer method is:


array([[1, 0, 1, 1, 1, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 1, 0, 1, 1]], dtype=int8)

In [46]:
final_model.get_best_log_evidence()

-3300.395469673639

In [47]:
final_model.get_best_log_evidence_icc()

array([-1754.41166022, -1545.98380945])

#### Simulated annealing

The simulated annealing algorithm has several parameters that can be set.

- SA_init_temperature : The initial annealing temperature (default = 100)
- SA_max_iterations : The maximum number of iterations after which the algorithms stops (default = 50 000)
- SA_update_schedule : The number of iterations before updating the temperature (default = 100)

In [50]:
final_model = searcher.simulated_annealing(data)

best log-evidence: -4317.31	@T = 100	i = 0
best log-evidence: -4025.41	@T = 100	i = 1
best log-evidence: -3503.42	@T = 100	i = 2
best log-evidence: -3305.55	@T = 100	i = 3
best log-evidence: -3300.4	@T = 14.9096	i = 304

- maximum iterations without improvement reached


In [51]:
print("The optimal partition of the variables found by the simulated annealing algorithm is:")
final_model.array

The optimal partition of the variables found by the simulated annealing algorithm is:


array([[1, 0, 1, 1, 1, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 1, 0, 1, 1]], dtype=int8)

In [52]:
final_model.get_best_log_evidence()

-3300.395469673639

In [53]:
init_model = searcher.get_model_in()
print("The starting partition of the algorithm is:")
init_model.array

The starting partition of the algorithm is:


array([[0, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0],
       [1, 0, 1, 0, 0, 0, 0, 0, 1]], dtype=int8)

#### Exhaustive search

The exhaustive search involves generating all possible partitions and calculating their log evidence. As this algorithm goes through all possible partitions, it is guaranteed to find the best one. For a system with $n$ variables, the number of different partitions is given by the Bell number of $n$, making this approach unfeasible for large systems ($n > 15$).

Because it goes through all possible partitions, the starting partition is irrelevant and no Model object should be given as input parameter

In [54]:
final_model = searcher.exhaustive_search(data)

In [55]:
print("The optimal partition of the variables found by the simulated annealing algorithm is:")
final_model.array

The optimal partition of the variables found by the simulated annealing algorithm is:


array([[1, 0, 1, 1, 1, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 1, 0, 1, 1]], dtype=int8)

In [56]:
searcher.log_evidence_trajectory

array([-3305.54657591, -3539.81806525, -3567.68653215, ...,
       -5202.81873422, -4977.24745505, -5258.10024044])